In [16]:
import numpy as np
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import normalized_mutual_info_score
import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN
from sklearn.metrics import normalized_mutual_info_score, adjusted_rand_score

from sklearn.metrics import (
    normalized_mutual_info_score,
    adjusted_rand_score,
    homogeneity_score,
    completeness_score,
    v_measure_score,
    fowlkes_mallows_score,
    silhouette_score
)
from scipy.optimize import linear_sum_assignment
import pandas as pd





In [17]:
def load_txt_data(file_path):
    data = np.loadtxt(file_path)
    X = data[:, :2]  # The first two columns are the features
    return X

def calculate_relative_density(X, k):
    nbrs = NearestNeighbors(n_neighbors=k+1, algorithm='auto').fit(X)
    distances, indices = nbrs.kneighbors(X)
    rd = np.sum(distances[:, 1:], axis=1) / k
    return rd

def classify_density(X, rd, t):
    high_density = X[rd < t]
    low_density = X[rd >= t]
    return high_density, low_density

def plot_density_classification(high_density, low_density):
    plt.figure(figsize=(10, 6))
    plt.scatter(high_density[:, 0], high_density[:, 1], color='blue', label='High Density')
    plt.scatter(low_density[:, 0], low_density[:, 1], color='red', label='Low Density')
    plt.legend()
    plt.title('Density Classification')
    plt.xlabel('Feature 1')
    plt.ylabel('Feature 2')
    plt.show()

def snnc(low_density, k):
    num = len(low_density)
    if num <= k:
        return low_density, np.array([])

    nbrs = NearestNeighbors(n_neighbors=min(k+1, num), algorithm='auto').fit(low_density)
    indices = nbrs.kneighbors(low_density, return_distance=False)
    clusters = [set([i]) for i in range(num)]

    for i in range(num):
        for j in range(i+1, num):
            if len(set(indices[i][1:]) & set(indices[j][1:])) >= 1:
                clusters[i] = clusters[i] | clusters[j]
                clusters[j] = set()

    merged = True
    while merged:
        merged = False
        for i in range(num):
            for j in range(i+1, num):
                if len(clusters[i] & clusters[j]) >= 1:
                    clusters[i] = clusters[i] | clusters[j]
                    clusters[j] = set()
                    merged = True

    clusters = [cluster for cluster in clusters if len(cluster) > 0]
    cluster_sizes = [len(cluster) for cluster in clusters]
    mean_cluster_size = np.mean(cluster_sizes)
    large_clusters = [cluster for cluster in clusters if len(cluster) >= mean_cluster_size]

    merged_indices = [index for cluster in large_clusters for index in cluster]
    merged_points = low_density[merged_indices]

    remaining_indices = list(set(range(num)) - set(merged_indices))
    remaining_points = low_density[remaining_indices]

    return merged_points, remaining_points

def plot_points(points, title):
    plt.figure(figsize=(10, 6))
    plt.scatter(points[:, 0], points[:, 1], color='green')
    plt.title(title)
    plt.xlabel('Feature 1')
    plt.ylabel('Feature 2')
    plt.show()

def union_high_density_and_remaining_points(high_density, unnature):
    if unnature.size == 0:
        return high_density
    if high_density.size == 0:
        return unnature
    return np.vstack((high_density, unnature))

def union_of_clusters(points, labels, other_points):
    union_points = np.vstack((points, other_points))
    new_cluster_label = labels.max() + 1 if labels.size > 0 else 0
    other_points_labels = np.full(other_points.shape[0], new_cluster_label)
    union_labels = np.concatenate((labels, other_points_labels))
    return union_points, union_labels

def run_dbscan(union_points, eps, min_samples, plot_title='DBSCAN Clustering'):
    dbscan = DBSCAN(eps=eps, min_samples=min_samples)
    cluster_labels = dbscan.fit_predict(union_points)
    # plot_points_with_clusters(union_points, cluster_labels, plot_title)
    return cluster_labels


def assign_outliers_to_nearest_cluster(data, labels):
    non_outliers = data[labels != -1]
    non_outlier_labels = labels[labels != -1]
    if len(non_outliers) == 0:
        return labels

    nbrs = NearestNeighbors(n_neighbors=1).fit(non_outliers)
    assigned_clusters = {}

    def find_nearest_non_outlier(point, visited):
        _, indices = nbrs.kneighbors([point])
        nearest_label = non_outlier_labels[indices[0][0]]
        return nearest_label

    def assign_cluster_to_outlier(outlier_index, visited):
        if outlier_index in assigned_clusters:
            return assigned_clusters[outlier_index]

        visited.add(outlier_index)
        point = data[outlier_index]
        nbrs_all = NearestNeighbors(n_neighbors=2).fit(data)
        _, indices = nbrs_all.kneighbors([point])

        nearest_index = indices[0][1]
        if labels[nearest_index] == -1:
            if nearest_index in visited:
                cluster = find_nearest_non_outlier(point, visited)
            else:
                cluster = assign_cluster_to_outlier(nearest_index, visited)
        else:
            cluster = labels[nearest_index]

        assigned_clusters[outlier_index] = cluster
        return cluster

    for i in range(len(data)):
        if labels[i] == -1:
            visited = set()
            labels[i] = assign_cluster_to_outlier(i, visited)

    return labels


import numpy as np
import matplotlib.pyplot as plt

def plot_points_with_clusters(X, reordered_labels, title):
    """
    Plots the points with their respective clusters, with no grid lines or ticks.

    Parameters:
    - X: numpy array or list of shape (n_samples, n_features), the data points.
    - reordered_labels: list or numpy array of shape (n_samples,), the cluster labels for the points.
    - title: str, the title for the plot.
    """
    plt.figure(figsize=(10, 8))
    unique_labels = set(reordered_labels)

    # Plot each cluster with a different color
    for label in unique_labels:
        # Find points belonging to the current cluster
        cluster_points = X[np.where(reordered_labels == label)]

        # Handle noise points if present
        if label == -1:
            plt.scatter(cluster_points[:, 0], cluster_points[:, 1], label='Noise', color='k')
        else:
            plt.scatter(cluster_points[:, 0], cluster_points[:, 1], label=f'Cluster {label}')

    plt.title(title)
    plt.xlabel('Feature 1')
    plt.ylabel('Feature 2')

    # Hide grid lines
    plt.grid(False)

    # Hide ticks and tick labels on both axes
    plt.xticks([])
    plt.yticks([])

    plt.legend()
    plt.show()

def MDBSCAN(file_path, eps, minpts, k, t):
    # Get the dataset
    X = load_txt_data(file_path)

    # Extract high and low density points
    rd = calculate_relative_density(X, k)
    high_density, low_density = classify_density(X, rd, t)
    # plot_density_classification(high_density, low_density)

    # Get the natural clusters and unnature points from low-density points
    natural_clusters, unnature = snnc(low_density, k)
    # plot_points(natural_clusters, "Natural Clusters in Low Density Points")

    # Find union of unnature and high density (union_points)
    union_points = union_high_density_and_remaining_points(high_density, unnature)

    # Run DBSCAN on the union
    cluster_labels = run_dbscan(union_points, eps, minpts)

    # Find the union of natural clusters and the result of DBSCAN
    up, ulabels = union_of_clusters(union_points, cluster_labels, natural_clusters)

    # Assign outliers to nearest cluster
    new_labels = assign_outliers_to_nearest_cluster(up, ulabels)

    # Map new_labels back to the original order of X
    label_mapping = {tuple(up[i]): new_labels[i] for i in range(len(up))}
    reordered_labels = np.array([label_mapping[tuple(point)] for point in X])

    # Plot the result
    # plot_points_with_clusters(X, reordered_labels, "MDBSCAN")

    # Return the result
    return reordered_labels, X





In [4]:
y , x = MDBSCAN('Compound_fixed.txt' ,1.5 ,6, 5 , 2.1)


In [18]:
def load_txt_data(file_path, with_labels=False):
    """
    Loads a 3-column text file:
      - cols 0/1 → features (X)
      - col 2   → integer ground-truth labels (y)
    """
    data = np.loadtxt(file_path)
    X = data[:, :2]
    if with_labels:
        y = data[:, 2].astype(int)
        return X, y
    return X

def MDBSCAN_with_scores(file_path, eps, minpts, k, t):
    # Load both X and y_true
    X, y_true = load_txt_data(file_path, with_labels=True)

    # Run your MDBSCAN to get y_pred
    y_pred, _ = MDBSCAN(file_path, eps, minpts, k, t)

    # Compute metrics
    nmi = normalized_mutual_info_score(y_true, y_pred)
    ari = adjusted_rand_score(y_true, y_pred)
    return y_pred, nmi, ari

In [6]:
y_pred, nmi, ari = MDBSCAN_with_scores(
        'Compound_fixed.txt',
        eps=1.5,
        minpts=6,
        k=5,
        t=2.1)
print(f"NMI = {nmi:.4f}")
print(f"ARI = {ari:.4f}")

NMI = 0.9858
ARI = 0.9939


In [19]:
import numpy as np

# core clustering evaluation metrics
from sklearn.metrics import (
    normalized_mutual_info_score,
    adjusted_rand_score,
    homogeneity_score,
    completeness_score,
    v_measure_score,
    fowlkes_mallows_score,
    silhouette_score
)
# pairwise precision/recall
from sklearn.metrics.cluster import pair_confusion_matrix
# for purity / accuracy mapping
from scipy.optimize import linear_sum_assignment

def load_txt_data(file_path, with_labels=False):
    data = np.loadtxt(file_path)
    X = data[:, :2]
    if with_labels:
        y = data[:, 2].astype(int)
        return X, y
    return X

def clustering_purity(y_true, y_pred):
    contingency = {}
    for yt, yp in zip(y_true, y_pred):
        contingency.setdefault(yp, {}).setdefault(yt, 0)
        contingency[yp][yt] += 1
    return sum(max(counts.values()) for counts in contingency.values()) / len(y_true)

def clustering_accuracy(y_true, y_pred):
    labels_pred = np.unique(y_pred)
    labels_true = np.unique(y_true)
    cost = np.zeros((labels_pred.size, labels_true.size), dtype=int)
    for i, cp in enumerate(labels_pred):
        for j, ct in enumerate(labels_true):
            cost[i, j] = -np.sum((y_pred == cp) & (y_true == ct))
    row_ind, col_ind = linear_sum_assignment(cost)
    return -cost[row_ind, col_ind].sum() / y_true.size

def pairwise_precision_recall(y_true, y_pred):
    tn, fp, fn, tp = pair_confusion_matrix(y_true, y_pred).ravel()
    prec = tp / (tp + fp) if tp + fp else 0.0
    rec  = tp / (tp + fn) if tp + fn else 0.0
    f1   = 2 * prec * rec / (prec + rec) if prec + rec else 0.0
    return prec, rec, f1

def evaluate_clustering(X, y_true, y_pred):
    results = {
        'NMI': normalized_mutual_info_score(y_true, y_pred),
        'ARI': adjusted_rand_score(y_true, y_pred),
        'Homogeneity': homogeneity_score(y_true, y_pred),
        'Completeness': completeness_score(y_true, y_pred),
        'V-measure': v_measure_score(y_true, y_pred),
        'Fowlkes–Mallows': fowlkes_mallows_score(y_true, y_pred),
        'Purity': clustering_purity(y_true, y_pred),
        'Accuracy': clustering_accuracy(y_true, y_pred)
    }
    results['Silhouette'] = (
        silhouette_score(X, y_pred)
        if len(set(y_pred)) > 1 else np.nan
    )
    prec, rec, f1 = pairwise_precision_recall(y_true, y_pred)
    results['Pairwise Precision'] = prec
    results['Pairwise Recall']    = rec
    results['Pairwise F1']        = f1
    return results

def MDBSCAN_with_full_evaluation(file_path, eps, minpts, k, t):
    X, y_true = load_txt_data(file_path, with_labels=True)
    y_pred, _ = MDBSCAN(file_path, eps, minpts, k, t)
    return evaluate_clustering(X, y_true, y_pred)

if __name__ == "__main__":
    scores = MDBSCAN_with_full_evaluation(
        'Compound_fixed.txt', eps=1.5, minpts=6, k=5, t=2.1
    )
    for metric, value in scores.items():
        print(f"{metric:20s}: {value:.4f}")


NMI                 : 0.9858
ARI                 : 0.9939
Homogeneity         : 0.9791
Completeness        : 0.9926
V-measure           : 0.9858
Fowlkes–Mallows     : 0.9954
Purity              : 0.9925
Accuracy            : 0.9925
Silhouette          : 0.1701
Pairwise Precision  : 0.9927
Pairwise Recall     : 0.9981
Pairwise F1         : 0.9954


In [8]:

# if __name__ == "__main__":
#     scores = MDBSCAN_with_full_evaluation(
#         'jain.txt', eps=1.5, minpts=6, k=6, t=1.3
#     )
#     for metric, value in scores.items():
#         print(f"{metric:20s}: {value:.4f}")

In [ ]:

if __name__ == "__main__":
    # Define your search grid
    eps_values    = [0.5, 1.0, 1.5, 2.0]
    minpts_values = [3, 5, 6, 8]
    k_values      = [3, 5, 6, 8]
    t_values      = [1.0, 1.3, 1.5, 2.1]

    # Storage for results
    records = []

    for eps in eps_values:
        for minpts in minpts_values:
            for k in k_values:
                for t in t_values:
                    scores = MDBSCAN_with_full_evaluation(
                        'jain.txt',
                        eps=eps,
                        minpts=minpts,
                        k=k,
                        t=t
                    )
                    # Flatten your record into a dict
                    rec = {
                        'eps': eps,
                        'minpts': minpts,
                        'k': k,
                        't': t,
                        **scores
                    }
                    records.append(rec)
                    print(f"Done eps={eps}, minpts={minpts}, k={k}, t={t}")

    # Build a DataFrame of results
    df = pd.DataFrame(records)

    # Sort by ARI (descending) and show the top few combos
    best = df.sort_values('ARI', ascending=False).head(10)
    print("\nTop 10 parameter sets by ARI:")
    print(best.to_string(index=False))

In [10]:
# if __name__ == "__main__":
#     scores = MDBSCAN_with_full_evaluation(
#         'jain.txt', eps=2.2, minpts=14, k=38, t=0.3
#     )
#     for metric, value in scores.items():
#         print(f"{metric:20s}: {value:.4f}")

# jain dataset

In [ ]:

if __name__ == "__main__":
    # Define your search grid
    eps_values    = [1,1.1,1.2,1.3,1.4,1.5,1.6,1.7,1.8,1.9,2]
    minpts_values = [1,2,3,4,5,6,7,8,9,10]
    k_values      = [1,2,3,4,5,6,7,8,9,10]
    t_values      = [0.8,0.9,1.0,1.1 , 1.2, 1.3,1.4, 1.5, 1.6,1.7,1.8,1.9,2,2.1,2.2,2.3,2.4]

    # Storage for results
    records = []

    for eps in eps_values:
        for minpts in minpts_values:
            for k in k_values:
                for t in t_values:
                    scores = MDBSCAN_with_full_evaluation(
                        'jain.txt',
                        eps=eps,
                        minpts=minpts,
                        k=k,
                        t=t
                    )
                    # Flatten your record into a dict
                    rec = {
                        'eps': eps,
                        'minpts': minpts,
                        'k': k,
                        't': t,
                        **scores
                    }
                    records.append(rec)
                    print(f"Done eps={eps}, minpts={minpts}, k={k}, t={t}")

    # Build a DataFrame of results
    df = pd.DataFrame(records)

    # Sort by ARI (descending) and show the top few combos
    best = df.sort_values('ARI', ascending=False).head(10)
    print("\nTop 10 parameter sets by ARI:")
    print(best.to_string(index=False))

Done eps=1, minpts=1, k=1, t=0.8
Done eps=1, minpts=1, k=1, t=0.9
Done eps=1, minpts=1, k=1, t=1.0
Done eps=1, minpts=1, k=1, t=1.1
Done eps=1, minpts=1, k=1, t=1.2
Done eps=1, minpts=1, k=1, t=1.3
Done eps=1, minpts=1, k=1, t=1.4
Done eps=1, minpts=1, k=1, t=1.5
Done eps=1, minpts=1, k=1, t=1.6
Done eps=1, minpts=1, k=1, t=1.7
Done eps=1, minpts=1, k=1, t=1.8
Done eps=1, minpts=1, k=1, t=1.9
Done eps=1, minpts=1, k=1, t=2
Done eps=1, minpts=1, k=1, t=2.1
Done eps=1, minpts=1, k=1, t=2.2
Done eps=1, minpts=1, k=1, t=2.3
Done eps=1, minpts=1, k=1, t=2.4
Done eps=1, minpts=1, k=2, t=0.8
Done eps=1, minpts=1, k=2, t=0.9
Done eps=1, minpts=1, k=2, t=1.0
Done eps=1, minpts=1, k=2, t=1.1
Done eps=1, minpts=1, k=2, t=1.2
Done eps=1, minpts=1, k=2, t=1.3
Done eps=1, minpts=1, k=2, t=1.4
Done eps=1, minpts=1, k=2, t=1.5
Done eps=1, minpts=1, k=2, t=1.6
Done eps=1, minpts=1, k=2, t=1.7
Done eps=1, minpts=1, k=2, t=1.8
Done eps=1, minpts=1, k=2, t=1.9
Done eps=1, minpts=1, k=2, t=2
Done eps=1, mi

# synthetic_dataset

In [ ]:

if __name__ == "__main__":
    # Define your search grid
    eps_values    = [0.1,0.2,0.3,0.4,0.5,0.6,0.7]
    minpts_values = [5,6,7,8,9,10,11,12]
    k_values      = [45,48,50,52,54]
    t_values      = [0.3,0.4,0.5,0.6,0.7,0.8,0.9]

    # Storage for results
    records = []

    for eps in eps_values:
        for minpts in minpts_values:
            for k in k_values:
                for t in t_values:
                    scores = MDBSCAN_with_full_evaluation(
                        'synthetic_dataset (3).txt',
                        eps=eps,
                        minpts=minpts,
                        k=k,
                        t=t
                    )
                    # Flatten your record into a dict
                    rec = {
                        'eps': eps,
                        'minpts': minpts,
                        'k': k,
                        't': t,
                        **scores
                    }
                    records.append(rec)
                    print(f"Done eps={eps}, minpts={minpts}, k={k}, t={t}")

    # Build a DataFrame of results
    df = pd.DataFrame(records)

    # Sort by ARI (descending) and show the top few combos
    best = df.sort_values('ARI', ascending=False).head(10)
    print("\nTop 10 parameter sets by ARI:")
    print(best.to_string(index=False))

Done eps=0.1, minpts=5, k=45, t=0.3
Done eps=0.1, minpts=5, k=45, t=0.4
Done eps=0.1, minpts=5, k=45, t=0.5
Done eps=0.1, minpts=5, k=45, t=0.6
Done eps=0.1, minpts=5, k=45, t=0.7
Done eps=0.1, minpts=5, k=45, t=0.8
Done eps=0.1, minpts=5, k=45, t=0.9
Done eps=0.1, minpts=5, k=48, t=0.3
Done eps=0.1, minpts=5, k=48, t=0.4
Done eps=0.1, minpts=5, k=48, t=0.5
Done eps=0.1, minpts=5, k=48, t=0.6
Done eps=0.1, minpts=5, k=48, t=0.7
Done eps=0.1, minpts=5, k=48, t=0.8
Done eps=0.1, minpts=5, k=48, t=0.9
Done eps=0.1, minpts=5, k=50, t=0.3
Done eps=0.1, minpts=5, k=50, t=0.4
Done eps=0.1, minpts=5, k=50, t=0.5
Done eps=0.1, minpts=5, k=50, t=0.6
Done eps=0.1, minpts=5, k=50, t=0.7
Done eps=0.1, minpts=5, k=50, t=0.8
Done eps=0.1, minpts=5, k=50, t=0.9
Done eps=0.1, minpts=5, k=52, t=0.3
Done eps=0.1, minpts=5, k=52, t=0.4
Done eps=0.1, minpts=5, k=52, t=0.5
Done eps=0.1, minpts=5, k=52, t=0.6
Done eps=0.1, minpts=5, k=52, t=0.7
Done eps=0.1, minpts=5, k=52, t=0.8
Done eps=0.1, minpts=5, k=52

# isolation

In [ ]:

if __name__ == "__main__":
    # Define your search grid
    eps_values    = [8,9,9.5,10,10.5,11]
    minpts_values = [2,3,4,5,6]
    k_values      = [2,3,4,5,6]
    t_values      = [8,9,10,11,12]


    # Storage for results
    records = []

    for eps in eps_values:
        for minpts in minpts_values:
            for k in k_values:
                for t in t_values:
                    scores = MDBSCAN_with_full_evaluation(
                        'isolation.txt',
                        eps=eps,
                        minpts=minpts,
                        k=k,
                        t=t
                    )
                    # Flatten your record into a dict
                    rec = {
                        'eps': eps,
                        'minpts': minpts,
                        'k': k,
                        't': t,
                        **scores
                    }
                    records.append(rec)
                    print(f"Done eps={eps}, minpts={minpts}, k={k}, t={t}")

    # Build a DataFrame of results
    df = pd.DataFrame(records)

    # Sort by ARI (descending) and show the top few combos
    best = df.sort_values('ARI', ascending=False).head(10)
    print("\nTop 10 parameter sets by ARI:")
    print(best.to_string(index=False))

Done eps=8, minpts=2, k=2, t=8
Done eps=8, minpts=2, k=2, t=9
Done eps=8, minpts=2, k=2, t=10
Done eps=8, minpts=2, k=2, t=11
Done eps=8, minpts=2, k=2, t=12
Done eps=8, minpts=2, k=3, t=8
Done eps=8, minpts=2, k=3, t=9
Done eps=8, minpts=2, k=3, t=10
Done eps=8, minpts=2, k=3, t=11
Done eps=8, minpts=2, k=3, t=12
Done eps=8, minpts=2, k=4, t=8
Done eps=8, minpts=2, k=4, t=9
Done eps=8, minpts=2, k=4, t=10
Done eps=8, minpts=2, k=4, t=11
Done eps=8, minpts=2, k=4, t=12
Done eps=8, minpts=2, k=5, t=8
Done eps=8, minpts=2, k=5, t=9
Done eps=8, minpts=2, k=5, t=10
Done eps=8, minpts=2, k=5, t=11
Done eps=8, minpts=2, k=5, t=12
Done eps=8, minpts=2, k=6, t=8
Done eps=8, minpts=2, k=6, t=9
Done eps=8, minpts=2, k=6, t=10
Done eps=8, minpts=2, k=6, t=11
Done eps=8, minpts=2, k=6, t=12
Done eps=8, minpts=3, k=2, t=8
Done eps=8, minpts=3, k=2, t=9
Done eps=8, minpts=3, k=2, t=10
Done eps=8, minpts=3, k=2, t=11
Done eps=8, minpts=3, k=2, t=12
Done eps=8, minpts=3, k=3, t=8
Done eps=8, minpts=3,

# compound

In [20]:

if __name__ == "__main__":
    # Define your search grid
    eps_values    = [0.9,1,1.1,1.2,1.3,1.4,1.5]
    minpts_values = [3,4,5,6]
    k_values      = [4,5,6]
    t_values      = [2,2.1,2.2]


    # Storage for results
    records = []

    for eps in eps_values:
        for minpts in minpts_values:
            for k in k_values:
                for t in t_values:
                    scores = MDBSCAN_with_full_evaluation(
                        'Compound_fixed.txt',
                        eps=eps,
                        minpts=minpts,
                        k=k,
                        t=t
                    )
                    # Flatten your record into a dict
                    rec = {
                        'eps': eps,
                        'minpts': minpts,
                        'k': k,
                        't': t,
                        **scores
                    }
                    records.append(rec)
                    print(f"Done eps={eps}, minpts={minpts}, k={k}, t={t}")

    # Build a DataFrame of results
    df = pd.DataFrame(records)

    # Sort by ARI (descending) and show the top few combos
    best = df.sort_values('ARI', ascending=False).head(10)
    print("\nTop 10 parameter sets by ARI:")
    print(best.to_string(index=False))

Done eps=0.9, minpts=3, k=4, t=2
Done eps=0.9, minpts=3, k=4, t=2.1
Done eps=0.9, minpts=3, k=4, t=2.2
Done eps=0.9, minpts=3, k=5, t=2
Done eps=0.9, minpts=3, k=5, t=2.1
Done eps=0.9, minpts=3, k=5, t=2.2
Done eps=0.9, minpts=3, k=6, t=2
Done eps=0.9, minpts=3, k=6, t=2.1
Done eps=0.9, minpts=3, k=6, t=2.2
Done eps=0.9, minpts=4, k=4, t=2
Done eps=0.9, minpts=4, k=4, t=2.1
Done eps=0.9, minpts=4, k=4, t=2.2
Done eps=0.9, minpts=4, k=5, t=2
Done eps=0.9, minpts=4, k=5, t=2.1
Done eps=0.9, minpts=4, k=5, t=2.2
Done eps=0.9, minpts=4, k=6, t=2
Done eps=0.9, minpts=4, k=6, t=2.1
Done eps=0.9, minpts=4, k=6, t=2.2
Done eps=0.9, minpts=5, k=4, t=2
Done eps=0.9, minpts=5, k=4, t=2.1
Done eps=0.9, minpts=5, k=4, t=2.2
Done eps=0.9, minpts=5, k=5, t=2
Done eps=0.9, minpts=5, k=5, t=2.1
Done eps=0.9, minpts=5, k=5, t=2.2
Done eps=0.9, minpts=5, k=6, t=2
Done eps=0.9, minpts=5, k=6, t=2.1
Done eps=0.9, minpts=5, k=6, t=2.2
Done eps=0.9, minpts=6, k=4, t=2
Done eps=0.9, minpts=6, k=4, t=2.1
Done 